In [39]:
import pandas as pd
import sqlalchemy
import requests
from config import ninja_api
import json

In [37]:
api_url = 'https://api.api-ninjas.com/v1/country?name='

In [69]:
owid_df = pd.read_csv('./Resources/owid-covid-data.csv')
owid_df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


In [74]:
red_owid_df = owid_df.copy()
red_owid_df = red_owid_df[['iso_code','continent', 'location', 'date', 'population', 
                           'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters']] 
                           #,'new_vaccinations', 'new_vaccinations_smoothed']]
red_owid_df

,iso_code,continent,location,date,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters
0,AFG,Asia,Afghanistan,2020-02-24,39835428.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,39835428.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,39835428.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,39835428.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,39835428.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
163782,ZWE,Africa,Zimbabwe,2022-02-18,15092171.0,7810556.0,4336161.0,3367921.0,106474.0
163783,ZWE,Africa,Zimbabwe,2022-02-19,15092171.0,7820885.0,4338919.0,3371563.0,110403.0
163784,ZWE,Africa,Zimbabwe,2022-02-20,15092171.0,7823894.0,4340412.0,3372616.0,110866.0
163785,ZWE,Africa,Zimbabwe,2022-02-21,15092171.0,7829096.0,4342121.0,3374371.0,112604.0


In [75]:
clean_df = red_owid_df.copy()
clean_df.dropna(inplace=True)
clean_df.reset_index(drop = True, inplace = True)
clean_df

,iso_code,continent,location,date,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters
0,ALB,Europe,Albania,2021-11-03,2872934.0,1926604.0,1025180.0,898625.0,2799.0
1,ALB,Europe,Albania,2021-11-06,2872934.0,1944676.0,1031588.0,904858.0,8230.0
2,ALB,Europe,Albania,2021-11-07,2872934.0,1947148.0,1032457.0,905876.0,8815.0
3,ALB,Europe,Albania,2021-11-09,2872934.0,1963359.0,1037546.0,910845.0,14968.0
4,ALB,Europe,Albania,2021-11-10,2872934.0,1971973.0,1040015.0,913524.0,18434.0
...,...,...,...,...,...,...,...,...,...
12019,ZWE,Africa,Zimbabwe,2022-02-18,15092171.0,7810556.0,4336161.0,3367921.0,106474.0
12020,ZWE,Africa,Zimbabwe,2022-02-19,15092171.0,7820885.0,4338919.0,3371563.0,110403.0
12021,ZWE,Africa,Zimbabwe,2022-02-20,15092171.0,7823894.0,4340412.0,3372616.0,110866.0
12022,ZWE,Africa,Zimbabwe,2022-02-21,15092171.0,7829096.0,4342121.0,3374371.0,112604.0


In [76]:
clean_country_df = clean_df.copy()
for n, row in clean_country_df.iterrows():
    row['location'] = row['location'].split('(', 1)[0]
clean_country_df

,iso_code,continent,location,date,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters
0,ALB,Europe,Albania,2021-11-03,2872934.0,1926604.0,1025180.0,898625.0,2799.0
1,ALB,Europe,Albania,2021-11-06,2872934.0,1944676.0,1031588.0,904858.0,8230.0
2,ALB,Europe,Albania,2021-11-07,2872934.0,1947148.0,1032457.0,905876.0,8815.0
3,ALB,Europe,Albania,2021-11-09,2872934.0,1963359.0,1037546.0,910845.0,14968.0
4,ALB,Europe,Albania,2021-11-10,2872934.0,1971973.0,1040015.0,913524.0,18434.0
...,...,...,...,...,...,...,...,...,...
12019,ZWE,Africa,Zimbabwe,2022-02-18,15092171.0,7810556.0,4336161.0,3367921.0,106474.0
12020,ZWE,Africa,Zimbabwe,2022-02-19,15092171.0,7820885.0,4338919.0,3371563.0,110403.0
12021,ZWE,Africa,Zimbabwe,2022-02-20,15092171.0,7823894.0,4340412.0,3372616.0,110866.0
12022,ZWE,Africa,Zimbabwe,2022-02-21,15092171.0,7829096.0,4342121.0,3374371.0,112604.0


In [78]:
countries = clean_country_df['iso_code'].unique()
len(countries)

116

In [79]:
countries_df = pd.DataFrame(columns = ['Country', 'GDP', 'Population Density', 'Urbanization', 'Unemployment'])
for nat in countries:
    response = requests.get(api_url+nat, headers={'X-Api-Key':ninja_api}).json()
    try:
        new_country_df = pd.DataFrame({'Country':[nat], 'GDP': [response[0]['gdp']], 
                                       'Population Density':[response[0]['pop_density']], 
                                       'Urbanization': [response[0]['urban_population']], 
                                       'Unemployment': [response[0]['unemployment']]})
        countries_df = pd.concat([countries_df, new_country_df], ignore_index = True, axis = 0)
    except (IndexError, KeyError):
        print(f'{nat} is not a valid country')
countries_df

AND is not a valid country
AIA is not a valid country
FRO is not a valid country
GIB is not a valid country
GRD is not a valid country
GGY is not a valid country
IMN is not a valid country
JEY is not a valid country
OWID_KOS is not a valid country
MCO is not a valid country
KNA is not a valid country
TWN is not a valid country


,Country,GDP,Population Density,Urbanization,Unemployment
0,ALB,15059.0,105.0,61.2,12.8
1,DZA,173757.0,18.4,73.2,11.5
2,ARG,518475.0,16.5,92.0,10.4
3,ARM,12433.0,104.1,63.2,16.6
4,AUS,1453871.0,3.3,86.1,5.3
...,...,...,...,...,...
99,USA,20580223.0,36.2,82.5,3.9
100,URY,59597.0,19.8,95.4,8.8
101,VEN,208338.0,32.2,88.2,9.4
102,VNM,244901.0,313.9,36.6,2.0
